In [ ]:
import os

print(os.getcwd())

In [ ]:
import glob
import pandas as pd
file_list = glob.glob("*/*.csv")
file_list_summary_tests = [f for f in file_list if "T-test and F-test for similarity of cv folds " in f]
file_list_summary_tests

In [ ]:
# Read in matching files
file_dict = {}
for file_name in file_list_summary_tests:
    file = pd.read_csv(file_name, index_col = 0) 
    file_name = file_name.replace(".csv", "")
    # All files    
    file_name = file_name.replace("T-test and F-test for similarity of cv folds ", "")
    LA, model_id = file_name.split('/')  
    file['LA'] = LA
    file['model_id'] = model_id
    file_dict[file_name] = file

print(file_dict.keys())

results = pd.concat(file_dict.values(), axis = 0, ignore_index = True)
model_id = results['model_id'].str.split('_', expand = True)
results['rq'], results['cv'], results['data_type'] = model_id[0], model_id[1], model_id[2]

In [ ]:
# Take the column average of the splits
f_test_cols = [col for col in results.columns if 'F-test' in col]
results['Average p-value from F-tests'] = results[f_test_cols].mean(skipna = True, axis = 1)
t_test_cols = [col for col in results.columns if 'T-test' in col]
results['Average p-value from T-tests'] = results[t_test_cols].mean(skipna = True, axis = 1)

# Aggregate by cv
f_test_cv = results.groupby('cv')['Average p-value from F-tests'].mean()
t_test_cv = results.groupby('cv')['Average p-value from T-tests'].mean()
tests_cv = pd.concat([f_test_cv, t_test_cv], axis = 1)

# Aggregate by data_type
f_test_data_type = results.groupby('data_type')['Average p-value from F-tests'].mean()
t_test_data_type = results.groupby('data_type')['Average p-value from T-tests'].mean()
tests_data_type = pd.concat([f_test_data_type, t_test_data_type], axis = 1)


In [ ]:
f_test_cv_data_type = results.groupby(['cv','data_type'])['Average p-value from F-tests'].mean()
t_test_cv_data_type = results.groupby(['cv','data_type'])['Average p-value from T-tests'].mean()
tests_cv_data_type = pd.concat([f_test_cv_data_type, t_test_cv_data_type], axis = 1)

In [ ]:
tests_cv.reset_index(drop = False, inplace = True)
tests_cv['cv'].replace({'ss': 'Learning from all cases', 'ts': 'Learning only from earlier cases'}, inplace = True)
tests_cv.to_csv('Output/Average p-values for difference between train and test data - cross validation.csv')
tests_cv

In [ ]:
tests_data_type.reset_index(drop = False, inplace = True)
tests_data_type['data_type'].replace({'all': 'Structured and text data', 'str': 'Structured data only'}, inplace = True)
tests_data_type.to_csv('Output/Average p-values for difference between train and test data - data type.csv')
tests_data_type